In [24]:
from pathlib import Path
from models.transfersdataset import TransfersDataset
from models.xgboost import Xgboost as Model

In [25]:
import pandas as pd
import warnings

In [26]:
warnings.filterwarnings("ignore")

In [27]:
td = TransfersDataset(Path.cwd()/'../prepped/final-2.csv')
td.encode_last_positions()
td.encode()
td.drop()

In [28]:
mid_features = ['succssdefact90','defduels90','defduelswpct','aerduels90','aerduelspct','sltackles90','padjsltackles','shotblocks90',
                'intrcpt90','padjintrcpt','fouls90','goals90','xg90','assists90','crosses90','acccrossespct','crossesleft90',
                'acccrossesleftpct','crossesright90','acccrossesrightpct','crosses6yard90','dribbles90','succssdribblespct','offduels90',
                'offduelspct','touchesbox90','prgruns90','acclr90','rcvdpasses90','rcvdlongpasses90','foulsa90','passes90',
                'accpassespct','fwdpasses90','accfwdpassespct','bpasses90','accbpassespct','latpasses90','acclatpassespct','shortmedpasses90',
                'accshortmedpassespct','longpasses90','acclongpassesct','avgpasslen','avglongpasslen','xa90','shotassist90','secassist90',
                'thirdassist90','smartpasses90','accsmartpassespct','kp90','final3rdpasses90','accfinal3rdpassespct','penareapasses90',
                'accpenareapassespct','thrpasses90','accthrpassespct','deepcomp90','deepcompcrosses90','prpasses90','accprpassespct',
                'directfk90','directfkontrgtpct','corners90','pens','pencnvrspct']
mid_features2 = ['assists90', 'succssdribblespct', 'rcards90', 'hdrgoals90', 'pens', 'accthrpassespct', 'defduelswpct', 'directfkontrgtpct', 'goalcnvrsnpct', 'fwdpasses90', 'final3rdpasses90', 'accprpassespct', 
                 'accsmartpassespct', 'sattact90', 'acclr90', 'penareapasses90']
mid_features3 = ['goals90', 'xg90','assists90', 'xa90', 'kp90', 'thrpasses90', 'final3rdpasses90', 'penareapasses90']

In [29]:
model_scores = pd.DataFrame()
def pipeline(pos, features=None, full_feature_set=False):
    td = TransfersDataset(Path.cwd()/'../prepped/final-2.csv')
    td.filter_postion(pos)
    td.encode()
    td.drop()
    m = Model(data = td.data, features = features, full_feature_set=full_feature_set)
    m.train_test_split()
    m.scale()
    m.tune_hp()
    m.train()
    m.inverse_scale()
    global model_scores
    model_scores = pd.concat([model_scores, m.score(thresh=1.5)])
    # display(m.feature_importance())
    # m.plot_predictions()
    m.top_n_predictions(20)
    m.top_n_predictions(20, worst=True)
    m.calculate_effectiveness()

In [30]:
pipeline('MID', mid_features)

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'max_depth': 162, 'learning_rate': 0.18666592800513737, 'n_estimators': 746, 'subsample': 0.717315338564266, 'colsample_bytree': 0.9264241722681141, 'gamma': 0.46444309284493307, 'reg_alpha': 0.13754691478933173, 'reg_lambda': 0.7679005083081629}
thresh = 1.5
mae: 3.312377280704045
mape: 0.5391905264080237
mae_below_thresh: 1.1105954183850968
mape_above_thresh: 0.39364451889309326


In [31]:
pipeline('MID', mid_features)

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'max_depth': 9, 'learning_rate': 0.198258357381625, 'n_estimators': 500, 'subsample': 0.6823062495333253, 'colsample_bytree': 0.7706491395450709, 'gamma': 0.18024529290554048, 'reg_alpha': 0.714412938663589, 'reg_lambda': 0.0012759774563376736}
thresh = 1.5
mae: 3.526160544384334
mape: 0.5242416615875833
mae_below_thresh: 0.9319258003575458
mape_above_thresh: 0.4082954584480278


In [32]:
scores = pd.read_csv('xgboost_mid.csv').iloc[:,1:].sort_values('MSE')
scores.Features = scores.Features.str.split(', ')
feature_sets = scores.Features.head(10).to_dict()

In [33]:
for _, features in feature_sets.items():
    pipeline(pos='MID', features=features, full_feature_set=True)

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'max_depth': 85, 'learning_rate': 0.18450078944880866, 'n_estimators': 477, 'subsample': 0.9064987165933008, 'colsample_bytree': 0.856293556369244, 'gamma': 0.18019756447262414, 'reg_alpha': 0.6485495772169794, 'reg_lambda': 0.8716873135710688}
thresh = 1.5
mae: 3.334493412405757
mape: 0.5188478227903065
mae_below_thresh: 0.9897598116738452
mape_above_thresh: 0.3935802036366427


  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'max_depth': 158, 'learning_rate': 0.1917445082089319, 'n_estimators': 1339, 'subsample': 0.7575028119212412, 'colsample_bytree': 0.8596999390989458, 'gamma': 0.4060257544381175, 'reg_alpha': 0.8304004488684127, 'reg_lambda': 0.904034991251497}
thresh = 1.5
mae: 3.4210971076609726
mape: 0.5526512232058617
mae_below_thresh: 1.2025684411185125
mape_above_thresh: 0.3890031046933698


  0%|          | 0/50 [00:00<?, ?it/s]

...

Best parameters: {'max_depth': 191, 'learning_rate': 0.19446046300527667, 'n_estimators': 1500, 'subsample': 0.7600956487043489, 'colsample_bytree': 0.727986261441274, 'gamma': 0.24431802397835015, 'reg_alpha': 0.2954519420470101, 'reg_lambda': 0.4699172882358762}
thresh = 1.5
mae: 3.4330349277641807
mape: 0.584950222673727
mae_below_thresh: 1.223103949683053
mape_above_thresh: 0.42166308130662544


  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'max_depth': 17, 'learning_rate': 0.19842495385758946, 'n_estimators': 857, 'subsample': 0.6219434487260946, 'colsample_bytree': 0.9886093327700427, 'gamma': 0.47238210455288565, 'reg_alpha': 0.5161025829485508, 'reg_lambda': 0.16183165285929685}
thresh = 1.5
mae: 3.270312631534318
mape: 0.5499459546542884
mae_below_thresh: 1.1120575005667548
mape_above_thresh: 0.3984999403925075


  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'max_depth': 182, 'learning_rate': 0.1870981369956145, 'n_estimators': 301, 'subsample': 0.9301767428764833, 'colsample_bytree': 0.8614265762394347, 'gamma': 0.3291723907899052, 'reg_alpha': 0.16274850827764675, 'reg_lambda': 0.47493479085928775}
thresh = 1.5
mae: 3.4394801147913525
mape: 0.5492414826441537
mae_below_thresh: 1.0459063475472585
mape_above_thresh: 0.4099530076492638


In [35]:
with pd.option_context('display.max_colwidth', None):
    display(model_scores.sort_values('mape_above_thresh'))

,mae,mape,mae_below_thresh,mape_above_thresh,features
0,3.220707,0.524801,1.103922,0.379667,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, offduelspct, thirdassist90, kp90, pencnvrspct, rcards90, npgoals90, deepcomp90, pens, goals90]"
0,3.421097,0.552651,1.202568,0.389003,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, offduelspct, thirdassist90, kp90, pencnvrspct, rcards90, npgoals90, deepcomp90, pens, goals90, defduels90, fk90, hdrgoals90, xa90, intrcpt90, acclongpassesct, avglongpasslen, ycards90, accfwdpassespct, latpasses90, xg90, accshortmedpassespct, goalcnvrsnpct, shots90, directfk90, penareapasses90, shotassist90, corners90, ontrgtpct]"
0,3.338374,0.560036,1.263444,0.393393,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, offduelspct, thirdassist90, kp90, pencnvrspct, rcards90, npgoals90, deepcomp90, pens, goals90, defduels90, fk90, hdrgoals90, xa90, intrcpt90, acclongpassesct, avglongpasslen, ycards90, accfwdpassespct, latpasses90, xg90, accshortmedpassespct, goalcnvrsnpct]"
0,3.334493,0.518848,0.989760,0.393580,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, offduelspct, thirdassist90, kp90, pencnvrspct, rcards90, npgoals90, deepcomp90, pens, goals90, defduels90, fk90, hdrgoals90, xa90, intrcpt90, acclongpassesct, avglongpasslen, ycards90, accfwdpassespct, latpasses90, xg90]"
0,3.312377,0.539191,1.110595,0.393645,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, succssdefact90, defduels90, defduelswpct, aerduels90, aerduelspct, sltackles90, padjsltackles, shotblocks90, intrcpt90, padjintrcpt, fouls90, goals90, xg90, assists90, crosses90, acccrossespct, crossesleft90, acccrossesleftpct, crossesright90, acccrossesrightpct, crosses6yard90, dribbles90, succssdribblespct, offduels90, offduelspct, touchesbox90, prgruns90, acclr90, rcvdpasses90, rcvdlongpasses90, foulsa90, passes90, accpassespct, fwdpasses90, accfwdpassespct, bpasses90, accbpassespct, latpasses90, acclatpassespct, shortmedpasses90, accshortmedpassespct, longpasses90, acclongpassesct, avgpasslen, avglongpasslen, xa90, shotassist90, secassist90, thirdassist90, smartpasses90, accsmartpassespct, kp90, final3rdpasses90, accfinal3rdpassespct, penareapasses90, accpenareapassespct, thrpasses90, accthrpassespct, deepcomp90, deepcompcrosses90, prpasses90, accprpassespct, directfk90, directfkontrgtpct, corners90, pens, pencnvrspct]"
0,3.270313,0.549946,1.112058,0.398500,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, offduelspct, thirdassist90, kp90, pencnvrspct, rcards90, npgoals90, deepcomp90, pens, goals90, defduels90, fk90, hdrgoals90, xa90, intrcpt90, acclongpassesct, avglongpasslen, ycards90, accfwdpassespct, latpasses90, xg90, accshortmedpassespct, goalcnvrsnpct, shots90, directfk90, penareapasses90, shotassist90, corners90, ontrgtpct, crosses90]"
0,3.526161,0.524242,0.931926,0.408295,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, succssdefact90, defduels90, defduelswpct, aerduels90, aerduelspct, sltackles90, padjsltackles, shotblocks90, intrcpt90, padjintrcpt, fouls90, goals90, xg90, assists90, crosses90, acccrossespct, crossesleft90, acccrossesleftpct, crossesright90, acccrossesrightpct, crosses6yard90, dribbles90, succssdribblespct, offduels90, offduelspct, touchesbox90, prgruns90, acclr90, rcvdpasses90, rcvdlongpasses90, foulsa90, passes90, accpassespct, fwdpasses90, acc